### Retrieve from MountainHub

This code allow for direct access to the MountainHub API. Data are returned in a Pandas dataframe.

Some code is adopted from Don Setiawan's work stored in the [cso legacy api](https://github.com/communitysnowobs/cso-legacy-api).

A. Arendt 20191231


In [82]:
import sys
import time
import datetime

import pandas as pd
import geopandas as gpd
import requests

BASE_URL = 'https://api.mountainhub.com/timeline'
HEADER = { 'Accept-version': '1' }
ONE_MONTH = 2592000000

Load these [utility functions from the cso-legacy-api](https://github.com/communitysnowobs/cso-legacy-api/blob/master/src/common/utils.py).

Todo: move these to a script after testing.

In [62]:
def empty_cso_dataframe():
    return pd.DataFrame(columns = [
        'author_name',
        'id',
        'timestamp',
        'lat',
        'long',
        'snow_depth',
        'source',
        'elevation',
        'comment'
    ])

def date_to_timestamp(date):
    """Converts datetime object to unix timestamp.
    Keyword arguments:
    date -- Datetime object to convert
    """
    if date is None:
        return date
    return int(time.mktime(date.timetuple())) * 1000

def timestamp_to_date(timestamp):
    """Converts unix timestamp to datettime object.
    Keyword arguments:
    timestamp -- Timestamp to convert
    """
    if timestamp is None:
        return timestamp
    return datetime.fromtimestamp(timestamp / 1000)

In [78]:
now_ts = date_to_timestamp(datetime.datetime.now())
start_ts = date_to_timestamp(datetime.datetime(2019,10,1))

Load parsing function from the [MountainHub cso-legacy-api script](https://github.com/communitysnowobs/cso-webapp/blob/dev/src/csoapi/apps/cso/types/snowobs.py).

#### NOTE: the MountainHub API has different named fields than what we were previously using in the legacy script above.

The code below is an attempt to match what is now in the MountainHub API:

In [86]:
def parse_data(results):
    observations = []

    for idx, res in enumerate(results):
        obs_data = {}
        try:
            observation = res['observation']
            obs_data['obs_id'] = observation['_id']
            obs_data['timestamp'] = int(observation['reported_at'])
            obs_data['obs_type'] = observation['type']
            obs_data['comment'] = observation['description']
            if len(observation['details']) > 0:
                if observation['details'][0]:
                    if 'snowpack_depth' in observation['details'][0].keys():
                        obs_data['snow_depth'] = observation['details'][0]['snowpack_depth'] 
        except:    
            obs_data['obs_id'] = 'None'
            #obs_data['timestamp'] = 'None'
            obs_data['obs_type'] = 'None'
            #obs_data['snow_depth'] = some dummy value?
        finally:
            actor = res['actor']
            if 'full_name' in actor.keys():
                obs_data['author_name'] = actor['full_name']
            elif 'fullName' in actor.keys():
                obs_data['author_name'] = actor['fullName']
            obs_data['id'] = idx
            obs_data['lat'] = res['location']['coordinates'][1]
            obs_data['lng'] = res['location']['coordinates'][0]
            obs_data['source'] = 'MountainHub'
        observations.append(obs_data)

    df = pd.DataFrame.from_records(observations).dropna()

    return df[df['snow_depth'] != 'undefined']

In [87]:
def fetch_raw_data(min_timestamp, max_timestamp, is_raw_json=False):

    args = {
      'publisher': 'all',
      'limit': 10000,
      'since' : min_timestamp,
      'before' : max_timestamp,
    }

    response = requests.get(BASE_URL, params=args, headers=HEADER)
    data = response.json()
    results = data['results']
    return parse_data(results)

In [91]:
df = fetch_raw_data(start_ts, now_ts )

In [92]:
df[df['comment'].str.contains("SAAW2019")]

,obs_id,timestamp,obs_type,comment,snow_depth,author_name,id,lat,lng,source
73,5dfd7f9980fa53189311c454,1.576875e+12,snow_conditions,Camp lake\nSAAW2019,33,Lee House,73,57.040973,-135.082323,MountainHub
95,5df56a5717f81d6d7d0a016d,1.576365e+12,snow_conditions,Middle Sister\nSAAW2019,43,Lee House,95,57.087043,-135.259928,MountainHub
112,5dead519374042277dad40a0,1.575671e+12,snow_conditions,SAAW2019,36,Lee House,112,57.073668,-135.317918,MountainHub
177,5ddadde6281f54198817401c,1.574625e+12,snow_conditions,SAAW2019,45,Emily Baker,177,61.251822,-149.513328,MountainHub
179,5dd9d5aa1bbaa50474c07c9b,1.574557e+12,snow_conditions,SAAW2019,0,Emily Baker,179,61.235399,-149.617811,MountainHub
180,5dd9d5a600793c7c47e89afa,1.574556e+12,snow_conditions,"SAAW2019 3 cm total low density new fluff, may...",3,Emily Baker,180,61.233255,-149.579530,MountainHub
181,5dd9d1fc7a711870a3d9bccf,1.574555e+12,snow_conditions,SAAW2019,40,Emily Baker,181,61.246685,-149.534172,MountainHub
182,5dd9d21a1bbaa50474c07aca,1.574554e+12,snow_conditions,SAAW2019,5,Emily Baker,182,61.248577,-149.528851,MountainHub
183,5dd9c9d61bbaa50474c07789,1.574554e+12,snow_conditions,SAAW2019,16,Emily Baker,183,61.252118,-149.518262,MountainHub
184,5dd9c8e81bbaa50474c076e7,1.574553e+12,snow_conditions,SAAW2019,85,Emily Baker,184,61.253634,-149.508786,MountainHub
